In [1]:
%pip install neupy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
# USANDO O LVQ NA BASE DE DADOS DO DIABETES
# ESCOLHENDO VARIOS PROTOTIPOS PARA A BASE

import pandas
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from neupy import algorithms
from sklearn.metrics import accuracy_score
plt.rcParams['figure.figsize'] = [16, 10]

#definindo os nomes de cada coluna   
names = ['num-pregnant', 'glucose', 'diastolic', 'triceps-skin', 'insulin', 'body-mass', 'diabetes-pedigree', 'age', 'class']

#Fazendo o carregamento dos dados diretamente do UCI Machine Learning          
dataset = pandas.read_csv("pima-indians-diabetes.csv", names=names)

# print("Primeiros dados")
# print(dataset.head(5))

#divisao de dados atributos e classe
X = dataset.values[:, 0:len(dataset.values[0])-1] #caracteristicas
Y = dataset.values[:, len(dataset.values[0])-1] #classe

#usando o metodo para fazer uma unica divisao dos dados
X_train, X_test, y_train, y_test = train_test_split( X, Y, test_size = 0.25, random_state = 10)

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.25, random_state = 10)

ImportError: cannot import name 'MutableMapping' from 'collections' (/usr/lib/python3.10/collections/__init__.py)

In [ ]:
p_range = range(3, 80)

parameters_scores = {}

p_scores_train = []
# p_scores_train_full = []
p_scores_valid = []

# use iteration to caclulator different k in models, then return the average accuracy based on the cross validation
for p in p_range:
    lvqnet = algorithms.LVQ(n_inputs=7, n_classes=2, n_subclasses=p)
    scores = cross_val_score(lvqnet, X_train, y_train, cv=5, scoring='accuracy')
    mean_cross_val_score = scores.mean()
    p_scores_train.append(mean_cross_val_score)
    
    lvqnet.train(X_train, y_train, epochs=20)
    # train_score = accuracy_score(lvqnet.predict(X_train),y_train)
    validation_score = accuracy_score(lvqnet.predict(X_valid), y_valid)
    # p_scores_train_full.append(train_score)
    p_scores_valid.append(validation_score)

    # We store the parameters used under the average score of CV and validation set
    registered_scores = [mean_cross_val_score, validation_score]
    avg_score = round((sum(registered_scores) / len(registered_scores)), 6)
    parameters_scores[avg_score] = (p, registered_scores)

# plot to see clearly
plt.plot(p_range, p_scores_train)
#plt.plot(p_range, p_scores_train_full)
plt.plot(p_range, p_scores_valid)
plt.legend(('Score medio Treino CV', 'Conj. Validacao'),
           loc='upper center', shadow=True)
plt.xlabel('Number of centroids for LVQ')
plt.ylabel('Cross-Validated Accuracy')
plt.show()

In [ ]:
# we create an instance of Neighbours Classifier and fit the data, using the best parameters.
best_avg = max(parameters_scores.keys())
(best_subclasses_number, registered_scores) = parameters_scores[best_avg]

lvqnet = algorithms.LVQ(n_inputs=7, n_classes=2, n_subclasses=best_subclasses_number)
lvqnet.train(X_train, y_train, epochs=20)

print("Classificador com os melhores parametros encontrados:")
print("subclasses:", best_subclasses_number)
print("Cross Val Score:", registered_scores[0])
print("validation set Score:", registered_scores[1])

print("Acuracia de treinamento clf: %0.3f" %  accuracy_score(lvqnet.predict(X_train),y_train))
print("Acuracia de teste clf: %0.3f" %  accuracy_score(lvqnet.predict(X_test), y_test))